In [ ]:
# installing required SQL magic packages 
#!pip install sqlachemy
#!pip install ipython-sql
#!pip install pymysql

In [ ]:
##import necessary libraries
import sqlite3
from sqlalchemy import create_engine
import pandas as pd


In [ ]:
# connect to the sqlite database (it will be created since it doesn't exit)
%load_ext sql

engine = create_engine("sqlite:///bonga_ecommerce.db")
%config SQL.conn_name = 'engine'

conn = sqlite3.connect("bonga_ecommerce.db")
cursor = conn.cursor()

%sql sqlite:///bonga_ecommerce.db

# Create table  

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
# Data Definition Languages (DDL)
# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS products(
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price DECIMAL NOT NULL,
    category TEXT NOT NULL
)    
''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS customers(
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
)    
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders(
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)    
''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS orderitems(
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity product_id NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)    
''')







In [ ]:
%%sql
SELECT name FROM sqlite_master WHERE type in ('table','view') AND name NOT LIKE 'sqlite_%' ORDER BY 1

 * sqlite:///bonga_ecommerce.db
Done.


name
customers
orderitems
orders
products


In [ ]:
# Drop tables
cursor.execute('''
DROP TABLE IF EXISTS products;
''')

cursor.execute('''
DROP TABLE IF EXISTS customers;
''')

cursor.execute('''
DROP TABLE IF EXISTS orders;
''')

cursor.execute('''
DROP TABLE IF EXISTS ordersitems;
''')

In [ ]:
#function to load the data into the database table
def load_data_to_table(csv_path, table_name):
    df = pd.read_csv(csv_path)
    df.to_sql(table_name, conn, if_exists= 'append', index=False)

In [ ]:
# loading the dataset to our database table
load_data_to_table('Products.csv', 'products')
load_data_to_table('Customers.csv', 'customers')
load_data_to_table('Orders.csv', 'orders')
load_data_to_table('Order_items.csv', 'orderitems')  

IntegrityError: UNIQUE constraint failed: orderitems.order_item_id

# DATA MANIPULATION LANGUAGE

 ## INSERT
 Example1

# Adding a new product

In [ ]:
%%sql

INSERT INTO products (name, price, category) VALUES ('New gadget', 299.99, 'Electronics')

 * sqlite:///bonga_ecommerce.db
1 rows affected.


[]

In [ ]:
%%sql

SELECT * FROM products; 

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
1,Product 1,193.52,Fashion
2,Product 2,475.85,Home & Kitchen
3,Product 3,368.68,Toys
4,Product 4,303.34,Electronics
5,Product 5,86.45,Toys
6,Product 6,86.44,Electronics
7,Product 7,38.46,Books
8,Product 8,434.43,Electronics
9,Product 9,304.55,Fashion
10,Product 10,356.96,Home & Kitchen


In [ ]:
cursor.execute("INSERT INTO products (name, price, category) VALUES (?,?,?)", ('New laptop', 399.99, 'Electronics'))

conn.commit()

In [ ]:
%%sql

SELECT * FROM products; 

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
1,Product 1,193.52,Fashion
2,Product 2,475.85,Home & Kitchen
3,Product 3,368.68,Toys
4,Product 4,303.34,Electronics
5,Product 5,86.45,Toys
6,Product 6,86.44,Electronics
7,Product 7,38.46,Books
8,Product 8,434.43,Electronics
9,Product 9,304.55,Fashion
10,Product 10,356.96,Home & Kitchen


# UPDATE

In [ ]:
%%sql

SELECT * FROM customers; 

 * sqlite:///bonga_ecommerce.db
Done.


customer_id,name,email
1,Customer1,Customer1@hotmail.com
2,Customer2,Customer2@gmail.com
3,Customer3,Customer3@gmail.com
4,Customer4,Customer4@hotmail.com
5,Customer5,Customer5@yahoo.com
6,Customer6,Customer6@gmail.com
7,Customer7,Customer7@gmail.com
8,Customer8,Customer8@gmail.com
9,Customer9,Customer9@hotmail.com
10,Customer10,Customer10@gmail.com


In [ ]:
%%sql

UPDATE customers SET email = 'Customer1@example.com' WHERE name = 'Customer1';


 * sqlite:///bonga_ecommerce.db
1 rows affected.


[]

In [ ]:
%%sql

SELECT * FROM customers; 

 * sqlite:///bonga_ecommerce.db
Done.


customer_id,name,email
1,Customer1,Customer1@example.com
2,Customer2,Customer2@gmail.com
3,Customer3,Customer3@gmail.com
4,Customer4,Customer4@hotmail.com
5,Customer5,Customer5@yahoo.com
6,Customer6,Customer6@gmail.com
7,Customer7,Customer7@gmail.com
8,Customer8,Customer8@gmail.com
9,Customer9,Customer9@hotmail.com
10,Customer10,Customer10@gmail.com


In [ ]:
%%sql

UPDATE customers SET name = 'Jane Doe' WHERE email = 'Customer2@gmail.com';

 * sqlite:///bonga_ecommerce.db
1 rows affected.


[]

In [ ]:
%%sql

SELECT * FROM customers; 

 * sqlite:///bonga_ecommerce.db
Done.


customer_id,name,email
1,Customer1,Customer1@example.com
2,Jane Doe,Customer2@gmail.com
3,Customer3,Customer3@gmail.com
4,Customer4,Customer4@hotmail.com
5,Customer5,Customer5@yahoo.com
6,Customer6,Customer6@gmail.com
7,Customer7,Customer7@gmail.com
8,Customer8,Customer8@gmail.com
9,Customer9,Customer9@hotmail.com
10,Customer10,Customer10@gmail.com


# DELETE 

In [ ]:
%%sql

SELECT * FROM orders; 

 * sqlite:///bonga_ecommerce.db
Done.


order_id,customer_id,order_date
1,168,2014-09-04 17:49:21
2,339,2011-07-19 03:46:15
3,193,2012-09-27 01:24:10
4,453,2020-03-23 10:56:21
5,283,2011-08-04 02:28:23
6,336,2023-11-17 16:37:04
7,390,2011-12-17 09:02:08
8,488,2019-01-27 06:59:07
9,13,2020-04-17 01:45:58
10,240,2012-07-04 01:37:27


In [ ]:
%%sql

    DELETE FROM orders WHERE order_id = 1; 

 * sqlite:///bonga_ecommerce.db
1 rows affected.


[]

In [ ]:
%%sql

SELECT * FROM orders; 

 * sqlite:///bonga_ecommerce.db
Done.


order_id,customer_id,order_date
2,339,2011-07-19 03:46:15
3,193,2012-09-27 01:24:10
4,453,2020-03-23 10:56:21
5,283,2011-08-04 02:28:23
6,336,2023-11-17 16:37:04
7,390,2011-12-17 09:02:08
8,488,2019-01-27 06:59:07
9,13,2020-04-17 01:45:58
10,240,2012-07-04 01:37:27
11,320,2011-01-06 19:27:40


In [ ]:
%%sql

    DELETE FROM products WHERE name = 'New gadget'; 

 * sqlite:///bonga_ecommerce.db
2 rows affected.


[]

In [ ]:
%%sql

SELECT * FROM products; 

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
1,Product 1,193.52,Fashion
2,Product 2,475.85,Home & Kitchen
3,Product 3,368.68,Toys
4,Product 4,303.34,Electronics
5,Product 5,86.45,Toys
6,Product 6,86.44,Electronics
7,Product 7,38.46,Books
8,Product 8,434.43,Electronics
9,Product 9,304.55,Fashion
10,Product 10,356.96,Home & Kitchen


# DATA QUERY LANGUAGE (DQL)

In [ ]:
%%sql

SELECT *
FROM products; 

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
1,Product 1,193.52,Fashion
2,Product 2,475.85,Home & Kitchen
3,Product 3,368.68,Toys
4,Product 4,303.34,Electronics
5,Product 5,86.45,Toys
6,Product 6,86.44,Electronics
7,Product 7,38.46,Books
8,Product 8,434.43,Electronics
9,Product 9,304.55,Fashion
10,Product 10,356.96,Home & Kitchen


In [ ]:
%%sql

SELECT *
FROM products
LIMIT 100; 

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
1,Product 1,193.52,Fashion
2,Product 2,475.85,Home & Kitchen
3,Product 3,368.68,Toys
4,Product 4,303.34,Electronics
5,Product 5,86.45,Toys
6,Product 6,86.44,Electronics
7,Product 7,38.46,Books
8,Product 8,434.43,Electronics
9,Product 9,304.55,Fashion
10,Product 10,356.96,Home & Kitchen


In [ ]:
%%sql

SELECT *
FROM products
LIMIT 30; 

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
1,Product 1,193.52,Fashion
2,Product 2,475.85,Home & Kitchen
3,Product 3,368.68,Toys
4,Product 4,303.34,Electronics
5,Product 5,86.45,Toys
6,Product 6,86.44,Electronics
7,Product 7,38.46,Books
8,Product 8,434.43,Electronics
9,Product 9,304.55,Fashion
10,Product 10,356.96,Home & Kitchen


In [ ]:
%%sql

SELECT product_id, name
FROM products
LIMIT 100; 

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name
1,Product 1
2,Product 2
3,Product 3
4,Product 4
5,Product 5
6,Product 6
7,Product 7
8,Product 8
9,Product 9
10,Product 10


In [ ]:
%%sql

SELECT product_id as 'ID', name as 'Product_Title'
FROM products
LIMIT 100; 

 * sqlite:///bonga_ecommerce.db
Done.


ID,Product_Title
1,Product 1
2,Product 2
3,Product 3
4,Product 4
5,Product 5
6,Product 6
7,Product 7
8,Product 8
9,Product 9
10,Product 10


# LIKE  OPERATOR

In [ ]:
%%sql

SELECT *
FROM customers
WHERE email LIKE '%@gmail.com' 

 * sqlite:///bonga_ecommerce.db
Done.


customer_id,name,email
2,Jane Doe,Customer2@gmail.com
3,Customer3,Customer3@gmail.com
6,Customer6,Customer6@gmail.com
7,Customer7,Customer7@gmail.com
8,Customer8,Customer8@gmail.com
10,Customer10,Customer10@gmail.com
14,Customer14,Customer14@gmail.com
17,Customer17,Customer17@gmail.com
18,Customer18,Customer18@gmail.com
19,Customer19,Customer19@gmail.com


In [ ]:
%%sql

SELECT *
FROM customers
WHERE email LIKE '@gmail.com%' 

 * sqlite:///bonga_ecommerce.db
Done.


customer_id,name,email


In [ ]:
%%sql

SELECT *
FROM customers
WHERE email LIKE '%@gmail.com%' 

 * sqlite:///bonga_ecommerce.db
Done.


customer_id,name,email
2,Jane Doe,Customer2@gmail.com
3,Customer3,Customer3@gmail.com
6,Customer6,Customer6@gmail.com
7,Customer7,Customer7@gmail.com
8,Customer8,Customer8@gmail.com
10,Customer10,Customer10@gmail.com
14,Customer14,Customer14@gmail.com
17,Customer17,Customer17@gmail.com
18,Customer18,Customer18@gmail.com
19,Customer19,Customer19@gmail.com


In [ ]:
%%sql

SELECT *
FROM products
WHERE name LIKE '%laptop%' 

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
None,New laptop,399.99,Electronics


In [ ]:
%%sql

INSERT INTO products (name, price, category) VALUES ('New gadget', 299.99, 'Electronics')

 * sqlite:///bonga_ecommerce.db
1 rows affected.


[]

In [ ]:
%%sql

SELECT *
FROM products
WHERE name LIKE '%gadget%' 

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
None,New gadget,299.99,Electronics


# BETWEEN